In [20]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB,GaussianNB

In [21]:
df = pd.read_csv('HR_comma_sep.csv')

In [22]:
X = df.drop('left',axis=1)

In [23]:
y = df['left']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=24, test_size=0.3, stratify=y)

In [31]:
ohe= OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
gb = GaussianNB()
pipe= Pipeline([('OHE', ohe), ('GB',gb)])
pipe.fit(X_train, y_train)
y_pred= pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.68      0.80      3429
           1       0.48      0.96      0.64      1070

    accuracy                           0.75      4499
   macro avg       0.73      0.82      0.72      4499
weighted avg       0.86      0.75      0.77      4499



In [32]:
y_pred_proba = pipe.predict_proba(X_test)[:,1]   #y=1 probability
print(roc_auc_score(y_test, y_pred_proba))

0.8318357985625628


In [33]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [37]:
params = {'GB__var_smoothing':np.linspace(1e-15,1,10)}

In [38]:
gcv = GridSearchCV(pipe,param_grid=params,scoring='roc_auc',cv=kfold)

In [39]:
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('GB', GaussianNB())]),
             param_grid={'GB__var_smoothing': array([1.00000000e-15, 1.11111111e-01, 2.22222222e-01, 3.33333333e-01,
       4.44444444e-01, 5.55555556e-01, 6.66666667e-01, 7.77777778e-01,
       8.88888889e-01, 1.00000000e+00])},
             scoring='roc_auc')

In [40]:
gcv.best_params_

{'GB__var_smoothing': 0.111111111111112}

In [42]:
gcv.best_score_

0.9654732081687083